In [ ]:
# Add project root to path
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

from scripts.download_google_buildings import (
    download_buildings,
    compute_building_counts,
    load_grid,
    visualize_building_counts,
    generate_tile_maps,
)
from src.utils.config_loader import load_config, get_data_dir

config = load_config()
data_dir = get_data_dir(config)
print(f"Data directory: {data_dir}")

In [ ]:
# Download buildings (only runs if file doesn't exist)
buildings_path = data_dir / "01_input_data" / "base_layers" / "overture_buildings.parquet"
output_dir = data_dir / "02_outputs" / "building_counts"
output_dir.mkdir(parents=True, exist_ok=True)

if not buildings_path.exists():
    grid = load_grid(data_dir, "5000", None, None)
    download_buildings(grid, buildings_path)
else:
    print(f"Buildings already downloaded: {buildings_path}")

In [ ]:
# Compute counts for all grid sizes
for grid_size in ["5000", "1000", "500"]:
    print(f"\n{'='*50}")
    print(f"Processing {grid_size}m grid")
    print("=" * 50)
    
    grid = load_grid(data_dir, grid_size, None, None)
    grid_with_counts = compute_building_counts(buildings_path, grid, grid_size)
    
    # Save
    label = "5km" if grid_size == "5000" else f"{grid_size}m"
    out_path = output_dir / f"grid_{label}_building_counts.gpkg"
    grid_with_counts.to_file(out_path, driver="GPKG")
    print(f"Saved: {out_path}")

In [ ]:
# Generate tile maps with Google satellite basemap for each grid cell
# This creates a PNG for each inhabited cell with buildings overlaid on satellite imagery

grid_size = "500"  # Change to "1000" or "5000" as needed
grid = load_grid(data_dir, grid_size, None, None)
grid_with_counts = compute_building_counts(buildings_path, grid, grid_size)

generate_tile_maps(
    grid_with_counts,
    buildings_path,
    output_dir,
    grid_size,
    basemap="hybrid",  # "satellite", "hybrid", or "roads"
    max_tiles=10,  # Set to None to generate all tiles
)